In [1]:
!pip install bitsandbytes==0.43.0
!pip install datasets==2.10.1
!pip install transformers==4.38.2
!pip install peft==0.9.0
!pip install sentencepiece==0.1.99
!pip install -U accelerate==0.28.0
!pip install colorama==0.4.6

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [2]:
import os
import sys
import argparse
import json
import warnings
import logging
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset, load_from_disk
import transformers
from datasets import Dataset
from peft import PeftModel
from colorama import *

from tqdm import tqdm
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig,DataCollatorForSeq2Seq, TrainingArguments, Trainer

from transformers import GenerationConfig
from peft import (
    prepare_model_for_int8_training,
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_kbit_training,
    TaskType
)
import random

In [3]:
train_dataset = Dataset.load_from_disk('autodl-tmp/dataset/datasets_train')
train_dataset

Dataset({
    features: ['instruction', 'input', 'response_total', 'response'],
    num_rows: 37123
})

In [4]:
val_dataset = Dataset.load_from_disk('autodl-tmp/dataset/datasets_val')
val_dataset

Dataset({
    features: ['instruction', 'input', 'response_total', 'response'],
    num_rows: 36869
})

In [5]:
tokenizer = AutoTokenizer.from_pretrained('autodl-tmp/qwen/Qwen2-7B-Instruct', trust_remote_code=True)
tokenizer

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Qwen2TokenizerFast(name_or_path='autodl-tmp/qwen/Qwen2-7B-Instruct', vocab_size=151643, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [6]:
prompt_lst = [
            "你是一个语义分类器。",
            "你将扮演一个文本意图识别器。",
            "你的功能是识别用户输入的语义类别。",
            "你的任务是将文本分配到相应的类别中。",
            "请将自己视为一个文本分类专家。",
            "你的目标是理解并分类文本的意图。",
            "你能够识别和归类不同的文本主题。",
            "你在这里是为了将文本信息分门别类。",
            "你的角色是文本意图的分类器。",
            "你是一个专业的语义识别系统。",
            "你专注于文本的语义分析和分类。",
            "你将提供文本意图的分类服务。",
            "你致力于精确地识别和分类文本内容。",
            "你的任务是分析文本并将其归入适当的类别。",
            "你将帮助识别文本的语义范畴。",
            "你能够快速地对文本进行语义分类。",
            "你的任务是将文本意图进行精确分类。",
            "你是文本语义分类助手。",
            "你将确保文本被正确地分类到其语义类别中。",
            "你的角色是确保文本分类的准确性和效率。",
            "你能为文本提供深入的语义分析和分类。"]  #len = 21

In [7]:
response_lst = [
    "A正面，B负面，C中性，D讽刺，E无产品评价语意",
    "A积极评价，B消极评价，C中立态度，D反讽意味，E无关产品评价的语境",
    "A正面反馈，B负面反馈，C中立观点，D讽刺语气，E非产品评价内容",
    "A正面情绪，B负面情绪，C无明显倾向，D讽刺性表达，E非评价性语义",
    "A赞赏性，B批评性，C客观性，D嘲讽性，E非评价性语义",
    "A正面评价倾向，B负面评价倾向，C评价中立，D讽刺性语气，E非评价性语境",
    "A积极情绪表达，B消极情绪表达，C情绪中性，D讽刺性语调，E非产品评价语义",
    "A正面情感色彩，B负面情感色彩，C情感中立，D讽刺性语气，E非评价性语义",
    "A正面评价，B负面评价，C评价中立，D讽刺性表达，E非产品评价语义",
    "A积极评价倾向，B消极评价倾向，C评价中性，D讽刺性语气，E非评价性语境",
    "A正面情感，B负面情感，C情感中立，D讽刺性语调，E非评价性语义",
    "A正面评价语境，B负面评价语境，C中立评价语境，D讽刺性语境，E非评价性语境",
    "A积极情绪倾向，B消极情绪倾向，C情绪中立，D讽刺性语气，E非产品评价语境",
    "A正面评价性质，B负面评价性质，C评价性质中立，D讽刺性质，E非评价性质",
    "A正面评价特征，B负面评价特征，C评价特征中立，D讽刺特征，E非评价特征",
    "A积极评价语义，B消极评价语义，C评价语义中立，D讽刺语义，E非评价语义",
    "A正面情感倾向性，B负面情感倾向性，C情感倾向性中立，D讽刺倾向性，E非评价倾向性",
    "A正面评价语义特征，B负面评价语义特征，C评价语义中立，D讽刺语义特征，E非评价语义特征",
    "A正面评价语境特征，B负面评价语境特征，C评价语境中立，D讽刺语境特征，E非评价语境特征",
    "A积极评价语义倾向，B消极评价语义倾向，C评价语义中立，D讽刺语义倾向，E非评价语义倾向",
    "A正面情感语义，B负面情感语义，C情感语义中立，D讽刺情感语义，E非情感评价语义"]
def reflash(x,label):
    start = x.find(label)
    end = x.find('，',start)
    return x[start:end]

In [8]:
def process_func_train(example):
    MAX_LENGTH = 386
    input_ids, attention_mask, labels = [], [], []
    instruction_seed = random.randint(0,len(prompt_lst)-1)
    select_seed = random.randint(0,15)
    instruct = prompt_lst[instruction_seed]
    select = response_lst[select_seed]
    get_select = example['response'][0]
    response_select = reflash(select, get_select)
    # 随机替换分类表现选项
    instruction = '<|im_start|>user\n' + instruct + (example['input']).replace("\'\'\'","").replace('简要说明理由','').replace(', "理由": " "','?') + '<|im_end|>\n'
    instruction = instruction.replace("A正面，B负面，C中性，D讽刺，E无产品评价语意",select)
    instruction = tokenizer(instruction, add_special_tokens = False)
    
    response = tokenizer(f'<|im_start|>assistant\n"分类标签":"{response_select}"<|im_end|>\n' , add_special_tokens = False)
    input_ids = instruction['input_ids'] + response['input_ids'] + [tokenizer.eos_token_id]
    attention_mask = instruction['attention_mask'] + response['attention_mask'] + [1]
    labels = [-100] * len(instruction['input_ids']) + response['input_ids'] + [tokenizer.eos_token_id]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

In [9]:
def process_func_val(example):
    MAX_LENGTH = 386
    input_ids, attention_mask, labels = [], [], []
    instruction_seed = random.randint(0,len(prompt_lst)-1)
    select_seed = random.randint(10,20)
    instruct = prompt_lst[instruction_seed]
    select = response_lst[select_seed]
    get_select = example['response'][0]
    response_select = reflash(select, get_select)
    # 随机替换分类表现选项
    instruction = '<|im_start|>user\n' + instruct + (example['input']).replace("\'\'\'","").replace('简要说明理由','').replace(', "理由": " "','?') + '<|im_end|>\n'
    instruction = instruction.replace("A正面，B负面，C中性，D讽刺，E无产品评价语意",select)
    instruction = tokenizer(instruction, add_special_tokens = False)
    
    response = tokenizer(f'<|im_start|>assistant\n"分类标签":"{response_select}"<|im_end|>\n' , add_special_tokens = False)
    input_ids = instruction['input_ids'] + response['input_ids'] + [tokenizer.eos_token_id]
    attention_mask = instruction['attention_mask'] + response['attention_mask'] + [1]
    labels = [-100] * len(instruction['input_ids']) + response['input_ids'] + [tokenizer.eos_token_id]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

In [10]:
tokenized_train = train_dataset.map(process_func_train, remove_columns=train_dataset.column_names)

In [11]:
tokenized_train_shuffle = tokenized_train.shuffle(seed=42)

In [12]:
tokenized_val = val_dataset.map(process_func_val, remove_columns=train_dataset.column_names)
tokenized_val

Map:   0%|          | 0/36869 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 36869
})

In [13]:
print(tokenizer.decode(tokenized_train_shuffle[0]['input_ids']))

<|im_start|>user
你将扮演一个文本意图识别器。#任务：以下是对卷烟产品的评论，按照分类标签内容语义进行评价，# 评论：review = "手上刚好有盒小字版软金沙。几个指标和网站不一样阿。12　 　 1.2　 　 　 　12。"# 输出格式： "分类标签": "A正面评价性质，B负面评价性质，C评价性质中立，D讽刺性质，E非评价性质"?<|im_end|>
<|im_start|>assistant
"分类标签":"B负面评价性质"<|im_end|>
<|im_end|>


In [14]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_train_shuffle[0]['labels'])))

'<|im_start|>assistant\n"分类标签":"B负面评价性质"<|im_end|>\n<|im_end|>'

## 创建模型

In [14]:
seed = 42
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

In [15]:
# 基础模型  16G+
model = AutoModelForCausalLM.from_pretrained(
    'autodl-tmp/qwen/Qwen2-7B',
    device_map='auto',
    torch_dtype=torch.bfloat16,
    # cache_dir=cache_dir,
    # quantization_config=nf4_config,
    # low_cpu_mem_usage = True
)
model

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Lin

In [16]:
# 开启检查点
model.enable_input_require_grads()

In [17]:
model.dtype

torch.bfloat16

## lora

In [18]:

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules = ['q_proj','k_proj','v_proj','o_proj','gate_proj','up_proj','down_proj'],
    inference_mode = False,
    r = 8,
    lora_alpha = 32,
    lora_dropout = 0.1
                            )
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'q_proj', 'down_proj', 'k_proj', 'o_proj', 'up_proj', 'gate_proj', 'v_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False)

In [19]:
peft_model = get_peft_model(model,config)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='autodl-tmp/qwen/Qwen2-7B', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'q_proj', 'down_proj', 'k_proj', 'o_proj', 'up_proj', 'gate_proj', 'v_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False)

In [20]:
# 可训练参数
peft_model.print_trainable_parameters()

trainable params: 20,185,088 || all params: 7,635,801,600 || trainable%: 0.26434798934534914


In [21]:
args = TrainingArguments(
    output_dir='autodl-tmp/peft/lora_f16',
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=100,
    num_train_epochs=3,
    save_steps=100,
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True,
    # shuffle=True
    
)

In [22]:
trainer = Trainer(
    model=peft_model,
    args=args,
    train_dataset=tokenized_train_shuffle,
    eval_dataset=tokenized_val,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
)

In [23]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
100,0.092800
200,0.036400
300,0.034400
400,0.034100
500,0.030300
600,0.033700
700,0.030800
800,0.029200
900,0.030900
1000,0.031000


Checkpoint destination directory autodl-tmp/peft/lora_f16/checkpoint-100 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory autodl-tmp/peft/lora_f16/checkpoint-200 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory autodl-tmp/peft/lora_f16/checkpoint-300 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory autodl-tmp/peft/lora_f16/checkpoint-400 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory autodl-tmp/peft/lora_f16/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory autodl-tmp/peft/lora_f16/checkpoint-600 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory autodl-

TrainOutput(global_step=6960, training_loss=0.022500848016519658, metrics={'train_runtime': 17067.3448, 'train_samples_per_second': 6.525, 'train_steps_per_second': 0.408, 'total_flos': 7.122248395331328e+17, 'train_loss': 0.022500848016519658, 'epoch': 3.0})

In [24]:
peft_model_path = 'autodl-tmp/peft/lora16'
trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

('autodl-tmp/peft/lora16/tokenizer_config.json',
 'autodl-tmp/peft/lora16/special_tokens_map.json',
 'autodl-tmp/peft/lora16/vocab.json',
 'autodl-tmp/peft/lora16/merges.txt',
 'autodl-tmp/peft/lora16/added_tokens.json',
 'autodl-tmp/peft/lora16/tokenizer.json')

In [22]:
peft_model.save_pretrained('peft_model_path')

In [23]:
trainer.model.save_pretrained('peft_model_path1')

In [ ]:
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=False,
   bnb_4bit_compute_dtype=torch.bfloat16
)




In [ ]:
peft_model = prepare_mod

In [10]:
# str1 = eval(data.iloc[0].system)['content']
# str2 = eval(data.iloc[0].user1)['content']
# s = str1 + str2
# prompt = s
# prompt = s
messages = [
   {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": train_dataset[0]['input']}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

In [11]:
text

'<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n#任务：以下是对卷烟产品的评论，按照分类标签内容语义进行评价，简要说明理由# 评论：review = "比五星红杉树要好抽\xa0一点，有点苏烟的味道"# 输出格式： "分类标签": "A正面，B负面，C中性，D讽刺，E无产品评价语意", "理由": " "<|im_end|>\n<|im_start|>assistant\n'

In [55]:
example = data.iloc[0]
instruction = '<|im_start|>user\n' + (eval(example['system'])['content'] + eval(example['user1'])['content']).replace("\'\'\'","").replace('简要说明理由','').replace(', "理由": " "','?') + '<|im_end|>\n'

instruction = tokenizer(instruction, add_special_tokens = False)
response = tokenizer('\n' + example['output'], add_special_tokens = False)

{'input_ids': [56568, 101909, 72881, 64559, 70538, 31548, 43326, 88802, 5122, 87752, 106273, 100199, 99752, 104703, 85641, 3837, 101892, 70538, 105151, 43815, 72881, 64559, 71817, 103964, 3837, 2, 220, 85641, 5122, 19417, 284, 330, 56006, 110937, 99425, 103268, 99613, 30534, 52801, 99950, 4102, 100380, 3837, 104037, 99908, 99752, 107254, 57676, 70568, 68805, 5122, 330, 70538, 105151, 788, 330, 32, 106557, 3837, 33, 103276, 3837, 34, 15946, 33071, 3837, 35, 115469, 3837, 36, 42192, 82700, 103964, 72881, 36589, 43869], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [54]:
lst = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
len(lst)

76

In [51]:
tokenizer.decode(instruction['input_ids'])

'你是一个语义分类器.#任务：以下是对卷烟产品的评论，按照分类标签内容语义进行评价，# 评论：review = "比五星红杉树要好抽\xa0一点，有点苏烟的味道"# 输出格式： "分类标签": "A正面，B负面，C中性，D讽刺，E无产品评价语意"?'

In [37]:
print(text)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
你是一个语义分类器.#任务：以下是对卷烟产品的评论，按照分类标签内容语义进行评价，简要说明理由# 评论：review = "比五星红杉树要好抽 一点，有点苏烟的味道"# 输出格式：''' "分类标签": "A正面，B负面，C中性，D讽刺，E无产品评价语意", "理由": " "<|im_end|>



In [23]:
data['total_tokens'].describe([0.999])

count    37123.000000
mean       302.579237
std         61.629622
min        177.000000
50%        293.000000
99.9%      697.878000
max       1221.000000
Name: total_tokens, dtype: float64

In [2]:
random.randint(5)

TypeError: Random.randint() missing 1 required positional argument: 'b'

In [ ]:
batchs = []
for i in range(len(data)):
    str1 = eval(data.iloc[i].system)['content']
    str2 = eval(data.iloc[i].user1)['content']
    s = str1 + str2
    prompt = s
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    batchs.append(text)

In [10]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]

In [17]:
text = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors='pt'

    )

tensor([[151644,   8948,    198,   2610,    525,    264,  10950,  17847,     13,
         151645,    198, 151644,    872,    198,  56568, 101909,  72881,  64559,
          70538,  31548,  43326,  88802,   5122,  87752, 106273, 100199,  99752,
         104703,  85641,   3837, 101892,  70538, 105151,  43815,  72881,  64559,
          71817, 103964,   3837,  98237,  30534,  66394, 102401,      2,    220,
          85641,   5122,  19417,    284,    330,  56006, 110937,  99425, 103268,
          99613,  30534,  52801,  99950,   4102, 100380,   3837, 104037,  99908,
          99752, 107254,  57676,  70568,  68805,   5122,  18788,    330,  70538,
         105151,    788,    330,     32, 106557,   3837,     33, 103276,   3837,
             34,  15946,  33071,   3837,     35, 115469,   3837,     36,  42192,
          82700, 103964,  72881,  36589,    497,    330, 102401,    788,    330,
            330, 151645,    198, 151644,  77091,    198]])

In [59]:
import datasets

In [26]:
data_for_dataset = data[['system','user1','assistant1','answer2']]

In [27]:
data_for_dataset.columns = ['instruction','input','response_total','response']

In [28]:
def format(x):
    return eval(x)['content'].replace('\'\'\'','')

In [32]:
data_for_dataset['response_total'] = data_for_dataset['response_total'].map(format)

/tmp/ipykernel_1431/3008227810.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_for_dataset['response_total'] = data_for_dataset['response_total'].map(format)


In [33]:
data_for_dataset

,instruction,input,response_total,response
0,你是一个语义分类器.,#任务：以下是对卷烟产品的评论，按照分类标签内容语义进行评价，简要说明理由# 评论：revi...,"""分类标签"": ""B负面"",""理由"": ""评论中的词汇'太难抽'、'又苦又涩'表达了消费者对...",B负面
1,你是一个语义分类器.,#任务：以下是对卷烟产品的评论，按照分类标签内容语义进行评价，简要说明理由# 评论：revi...,"""分类标签"": ""B负面"",""理由"": ""评论中明确指出产品的味道不好，性价比低，这些都是直...",B负面
2,你是一个语义分类器.,#任务：以下是对卷烟产品的评论，按照分类标签内容语义进行评价，简要说明理由# 评论：revi...,"""分类标签"": ""B负面"",""理由"": ""评论中明确表达了对于产品的不满，'没咖啡味，不好抽...",B负面
3,你是一个语义分类器.,#任务：以下是对卷烟产品的评论，按照分类标签内容语义进行评价，简要说明理由# 评论：revi...,"""分类标签"": ""B负面"",""理由"": ""评论中明确表示了对产品的不满，提到了‘毫无咖啡味’...",B负面
4,你是一个语义分类器.,#任务：以下是对卷烟产品的评论，按照分类标签内容语义进行评价，简要说明理由# 评论：revi...,"""分类标签"": ""B负面"",""理由"": ""评论中的“太TM难抽了”表达了强烈的负面情绪，'T...",B负面
...,...,...,...,...
36864,你是一个语义分类器.,#任务：以下是对卷烟产品的评论，按照分类标签内容语义进行评价，简要说明理由# 评论：revi...,"""分类标签"": ""A正面"",""理由"": ""评论者对于卷烟产品的描述带有明显的怀旧情感，提到了...",A正面
36865,你是一个语义分类器.,#任务：以下是对卷烟产品的评论，按照分类标签内容语义进行评价，简要说明理由# 评论：revi...,"""分类标签"": ""A正面"",""理由"": ""评论中提到‘特别好抽’，并且将产品与同等价位的其他...",A正面
36866,你是一个语义分类器.,#任务：以下是对卷烟产品的评论，按照分类标签内容语义进行评价，简要说明理由# 评论：revi...,"""分类标签"": ""A正面"",""理由"": ""评论中提到‘太想念这个味道了’和‘太棒了’，明显表...",A正面
36867,你是一个语义分类器.,#任务：以下是对卷烟产品的评论，按照分类标签内容语义进行评价，简要说明理由# 评论：revi...,"""分类标签"": ""B负面"",""理由"": ""评论中提到了‘比硬中华劲大’，这通常意味着香烟的口...",C中性


In [22]:
train_data = Dataset.from_pandas(data_for_dataset)

In [34]:
val_data = Dataset.from_pandas(data_for_dataset)

In [36]:
val_data.save_to_disk('./autodl-tmp/dataset/datasets_val')

Saving the dataset (0/1 shards):   0%|          | 0/36869 [00:00<?, ? examples/s]

In [40]:
Dataset.load_from_disk('./autodl-tmp/dataset/datasets_val')

Dataset({
    features: ['instruction', 'input', 'response_total', 'response'],
    num_rows: 36869
})